<a href="https://colab.research.google.com/github/saharshhhh/DeepLearning/blob/main/Topic_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-h

In [1]:
from gensim.models import LdaModel, CoherenceModel
from gensim.corpora import Dictionary
from nltk.corpus import stopwords
import nltk,re

In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from sklearn.datasets import fetch_20newsgroups
newsgroups=fetch_20newsgroups(remove=('headers','footers','quotes'))
documents=newsgroups.data

In [7]:
def clean_text(text):
  text=re.sub(r'\W+',' ' ,text.lower())
  tokens=[word for word in text.split() if word not in stop_words and len(word) > 3]
  return tokens

In [8]:
#tokenize all documents
tokenized_docs = [clean_text(doc) for doc in documents]

In [9]:
dictionary=Dictionary(tokenized_docs)
dictionary.filter_extremes(no_below=10,no_above=0.9)
corpus=[dictionary.doc2bow(doc) for doc in tokenized_docs]

In [10]:
lda_model = LdaModel(corpus=corpus,id2word=dictionary,num_topics=8,random_state=42,passes=15,alpha='auto', per_word_topics=True)

In [14]:
print('Top Words per Topic:')
for idx, topic in lda_model.print_topics(-1):
  # Extract words from the topic string
  words = [word.split('*')[1].strip().strip('"') for word in topic.split('+')]
  print(f"Topic {idx+1}: {'|'.join(words)}")

Top Words per Topic:
Topic 1: data|system|encryption|chip|keys|clipper|information|used|public|security
Topic 2: jesus|bible|christian|believe|church|faith|book|christ|christians|religion
Topic 3: year|game|team|first|time|last|good|like|back|games
Topic 4: file|windows|program|files|software|available|window|version|using|image
Topic 5: people|government|president|said|israel|armenian|armenians|turkish|jews|state
Topic 6: would|drive|like|know|anyone|card|thanks|system|also|need
Topic 7: space|1993|nasa|university|april|research|national|information|center|year
Topic 8: would|people|think|like|know|even|could|well|make|much


In [15]:
coherence_model=CoherenceModel(model=lda_model,texts=tokenized_docs,dictionary=dictionary,coherence='c_v')
coherence_score=coherence_model.get_coherence()
print(f'Coherence Score: {coherence_score}')

Coherence Score: 0.6056214829429873


In [17]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.1 MB/s eta 0:00:00


In [20]:
import pyLDAvis.gensim_models
import pickle
import pyLDAvis

vis_data=pyLDAvis.gensim_models.prepare(lda_model,corpus,dictionary,sort_topics=False)


In [21]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)